In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.atp-autoteile.de/de/product/vehicles/' + input_.loc[a, 'Product_Id'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_make = [make.strip() for make in html.xpath('//div[@data-manufacturer-selector]/text()')]
    
                # = = = = = = = = = = = = = = =
                
                list_make_code = [code.strip() for code in html.xpath('//div[@data-manufacturer-selector]/@data-tab')]
     
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'No.': int(input_.loc[a, '序号']),
                                        'Product_Id': input_.loc[a, 'Product_Id'],
                                        'Make': list_make,
                                        'Make_Code': list_make_code})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'序号': int(input_.loc[a, '序号']),
                                     'Product_Id': input_.loc[a, 'Product_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Product_Id'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Make_Code'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./make-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./make_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：150

[ok] - 4234520
[尝试次数：1] - [剩余数量：130] - [当前时间：14:01:35]

[ok] - 4158821
[尝试次数：1] - [剩余数量：129] - [当前时间：14:01:36]

[ok] - 3648983
[尝试次数：1] - [剩余数量：128] - [当前时间：14:01:36]

[ok] - 4098330
[尝试次数：1] - [剩余数量：127] - [当前时间：14:01:36]

[ok] - 1075829
[尝试次数：1] - [剩余数量：126] - [当前时间：14:01:36]

[ok] - 3648954
[尝试次数：1] - [剩余数量：125] - [当前时间：14:01:36]

[ok] - 4118028
[尝试次数：1] - [剩余数量：124] - [当前时间：14:01:36]

[ok] - 4570446
[尝试次数：1] - [剩余数量：123] - [当前时间：14:01:36]

[ok] - 4091955
[尝试次数：1] - [剩余数量：122] - [当前时间：14:01:36]

[ok] - 3642704
[尝试次数：1] - [剩余数量：121] - [当前时间：14:01:36]

[ok] - 1093509
[尝试次数：1] - [剩余数量：120] - [当前时间：14:01:36]

[ok] - 4316658
[尝试次数：1] - [剩余数量：119] - [当前时间：14:01:36]

[ok] - 4234508
[尝试次数：1] - [剩余数量：118] - [当前时间：14:01:37]

[ok] - 4316295
[尝试次数：1] - [剩余数量：117] - [当前时间：14:01:37]

[ok] - 1051673
[尝试次数：1] - [剩余数量：116] - [当前时间：14:01:37]

[ok] - 4310099
[尝试次数：1] - [剩余数量：115] - [当前时间：14:01:38]

[ok] - 937462
[尝试次数：1] - [剩余数量：114] - [当前时间：14:01:38]

[ok] - 1144742
[尝试次数：1] - [剩余数量：113] - [